### 資料準備  

In [ ]:
import pandas as pd
import gensim
import collections
import jieba, operator, os, json,re ,sys
# 這裡設定常數
jieba.set_dictionary('C:\Users\user\Anaconda2\Lib\site-packages\jieba\dict.txt.big.txt')
stopPath = 'chinese_stopword.txt'
modelName = u'watch1model'
path = u'C:\\watch\\'



# 讀入停止詞，裝入stop_list
stop_list = []
with open(stopPath) as s :
    
    lines = s.readlines()
    for line in lines:
        line = line.strip()
        stop_list.append(line)
#定義一個函數，進入一行一行的文字，output 切完詞後濾停止詞，在裝入list
def wordCut(text):
    cutlist = u"<>/:：;；,、＂’，.。！？｢\"\'\\r《》“”!@#$%^&*()★❤◆■"
    
    wlist = []
    words = jieba.cut(text, cut_all=False)
    wlist=(list(word.strip() for word in words if word not in set(stop_list)))
    
#     for word in words:
#         if word not in stop_list:
#             wlist.append(word)
    
    return wlist     


# 開始讀取檔案
fileList=os.listdir(path)
url = []
afterCut = []
for name in fileList:
    line =' '
    with open(path+name) as f :        
        # 逐行讀入，將yahoo結尾標改為_|統一格式
        while len(line)>0 : 
            line = f.readline()
            line = line.replace('$&','_|')
            
            
            if len(line) ==0:                
                f.close()                
            else:
                # 拆出標題和內文，如果內文出現 ); 則跳過此行
                try:
                    wholeContent=line.split('_|')
                    corrupt=re.findall('\)\;',wholeContent[-2])
                    if  len(corrupt)>0 :
                        continue
                    # 紀錄網址    
                    url.append(wholeContent[1])    
                    content = wholeContent[0]+wholeContent[-2]         
                    
                    #紀錄切詞結果   
                    afterCut.append(wordCut(content))
                except:
                    #不知道為什麼會超過範圍，香水約有5000多筆
                    print 'IndexError , list out of range'


Building prefix dict from C:\Users\user\Anaconda2\Lib\site-packages\jieba\dict.txt.big.txt ...
Loading model from cache c:\users\user\appdata\local\temp\jieba.uc36b00d9e531beb3c49093c3cd57785c.cache
Loading model cost 0.588 seconds.
Prefix dict has been built succesfully.


IndexError , list out of range
IndexError , list out of range
IndexError , list out of range
IndexError , list out of range
IndexError , list out of range
IndexError , list out of range
IndexError , list out of range
IndexError , list out of range
IndexError , list out of range
IndexError , list out of range
IndexError , list out of range
IndexError , list out of range
IndexError , list out of range
IndexError , list out of range
IndexError , list out of range
IndexError , list out of range
IndexError , list out of range
IndexError , list out of range
IndexError , list out of range
IndexError , list out of range
IndexError , list out of range
IndexError , list out of range
IndexError , list out of range
IndexError , list out of range
IndexError , list out of range
IndexError , list out of range
IndexError , list out of range
IndexError , list out of range
IndexError , list out of range
IndexError , list out of range
IndexError , list out of range
IndexError , list out of range
IndexErr

# 從網址中拆出商品編號，按照doc2vec規定格式存入list

In [ ]:
urlNo = []
for i in url :
    if 'show?' in i :
        urlNo.append(i.split('show?')[1])
    else:
        temp=i.split(';_')[0]
        
        
        urlNo.append(temp.split('/')[-1])
i=0
tagged =[]
for no in urlNo :    
    tagged.append(gensim.models.doc2vec.LabeledSentence(afterCut[i], [urlNo[i]]))
    i = i+1
#print tagged  

# 此block 訓練doc2vec

In [ ]:
model = gensim.models.doc2vec.Doc2Vec(size=400, min_count=1, iter=10,workers=12,window=10,alpha=0.025, min_alpha=0.025)
model.build_vocab(tagged)
for epoch in range(10):
    model.train(tagged)
    model.alpha -= 0.002 # decrease the learning rate
    model.min_alpha = model.alpha # fix the learning rate, no deca
    %time model.train(tagged)
model.save(modelName)

# 這個block訓練k-mean

In [ ]:
from sklearn.cluster import MiniBatchKMeans
import time

start = time.time() # Start time

doc_vectors = model.docvecs.doctag_syn0
num_clusters = 500
kmeans_clustering = MiniBatchKMeans( n_clusters = num_clusters,init_size=9000)
idx = kmeans_clustering.fit( doc_vectors )
end = time.time()
elapsed = end - start
print "Time taken for K Means clustering: ", elapsed, "seconds."

## (option) 觀察分群狀況，群編號:群內含有多少商品

In [ ]:
# kmeans_clustering.labels_ 為list 內含分群資訊，按照原始順序
print (kmeans_clustering.labels_)
count ={}
for i in range(len(kmeans_clustering.labels_)):
    whichCluster=kmeans_clustering.labels_[i]
    if whichCluster not in count:
        count[whichCluster] = 1
    else:
        count[whichCluster]+=1
       # no=model.docvecs.index_to_doctag(i)
print count

## 簡單測試機 

In [ ]:
dic = {'no':urlNo , 
       'content':afterCut,
       'cluster':kmeans_clustering.labels_.tolist(),
#        'aftercut' :wlist
      }
pdf = pd.DataFrame.from_dict(dic)
#df = DataFrame(d, index=['r', 'd', 'a'], columns=['two', 'three'])

a=pdf[pdf.no =='21312192523180'].values.tolist()
print ' '.join(a[0][0])
b=pdf[pdf.no =='21312192523180'].values.tolist()
print ' '.join(b[0][0])
        

In [7]:
a = [1,2,3]
with open('./testline.txt', 'w') as f:
    for n in a:
        f.write(str(n)+'\n')

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__delslice__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getslice__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__setslice__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']